In [2]:
import pandas as pd
import datetime

In [23]:
ibge = pd.read_csv('./dados/ibge_drs.csv')

#função para iterar datas
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(i)

inicio = datetime.datetime(2020,1,1)
fim = datetime.datetime.now()

for cod in ibge.iloc():
    cidade = str(cod['Código IBGE'])
    co = pd.read_csv('casos_obitos/' + cidade + '.csv')
    vacinacao = pd.read_csv('vacinacao/' + cidade + '.csv')
    isolamento = pd.read_csv('isolamento/datas_formatadas/' + cidade + '.csv')
    ocupacao = pd.read_csv('ocupacao/' + cidade + '.csv')
    
    try:
        r = pd.read_csv('r_efetivo/cidades/' + cidade + '.csv')
    except FileNotFoundError:
        pass
    
    
    
    dados = { 'Município': [], 'Código IBGE': [], 'Populacao': [], 'Total 1ª Dose': [],
       'Total 2ª Dose': [], 'Total Unica': [], 'Total Doses Aplicadas': [],
       'População Vacinada dose1/População Total': [],
       'População Vacinada/População Total': [], 'data': [], 'diagnostico_covid19': [], 'obito': [], 'media_movel_casos': [],
       'media_movel_obitos': [], 'ocupacao_leitos': []}
    novo = pd.DataFrame(data=dados)
    for data in daterange(inicio, fim):
        
        iso = isolamento[isolamento['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'DRS', 'cidade', 'cod_ibge'])
        vac = vacinacao[vacinacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0'])
        cao = co[co['data_inicio_sintomas'] == data.strftime('%Y-%m-%dT00:00:00Z')].drop(columns=['Unnamed: 0','nome_munic', 'codigo_ibge', 'data_inicio_sintomas'])
        cao['data'] = data.strftime('%Y-%m-%d')
        ocu = ocupacao[ocupacao['data'] == data.strftime('%Y-%m-%d')].drop(columns=['Unnamed: 0', 'cod_ibge', 'data'])
        ocu['data'] = data.strftime('%Y-%m-%d')
        
        soma = pd.merge(vac, cao, on="data")
        soma = pd.merge(soma, ocu, on="data")
        soma = pd.merge(soma, iso, on="data")
        
        if(not(vac.empty) and not(cao.empty) and not(ocu.empty)):
            novo = novo.append(soma)
    novo = novo.reset_index().drop(columns='index')
    break

#print(novo.dropna())
novo

,Município,Código IBGE,Populacao,Total 1ª Dose,Total 2ª Dose,Total Unica,Total Doses Aplicadas,População Vacinada dose1/População Total,População Vacinada/População Total,data,diagnostico_covid19,obito,media_movel_casos,media_movel_obitos,ocupacao_leitos,media_isolamento
0,ADAMANTINA,3500105.0,35111.0,2068.0,543.0,0.0,2611.0,5.889892,1.546524,2021-02-21,11.0,1.0,16.571429,0.571429,"79,81",43.00
1,ADAMANTINA,3500105.0,35111.0,2079.0,547.0,0.0,2626.0,5.921221,1.557916,2021-02-22,24.0,1.0,17.428571,0.571429,"80,29",37.25
2,ADAMANTINA,3500105.0,35111.0,2104.0,608.0,0.0,2712.0,5.992424,1.731651,2021-02-23,13.0,0.0,15.142857,0.571429,"80,77",38.25
3,ADAMANTINA,3500105.0,35111.0,2110.0,615.0,0.0,2725.0,6.009513,1.751588,2021-02-24,10.0,1.0,14.285714,0.571429,"79,81",50.25
4,ADAMANTINA,3500105.0,35111.0,2125.0,739.0,0.0,2864.0,6.052234,2.104753,2021-02-25,7.0,1.0,11.714286,0.714286,"77,03",38.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,ADAMANTINA,3500105.0,35111.0,21276.0,7906.0,632.0,29814.0,60.596394,24.317166,2021-07-15,9.0,0.0,10.428571,0.142857,"69,18",39.25
145,ADAMANTINA,3500105.0,35111.0,21559.0,7968.0,632.0,30159.0,61.402410,24.493748,2021-07-16,6.0,0.0,9.857143,0.142857,"72,95",37.75
146,ADAMANTINA,3500105.0,35111.0,21747.0,7997.0,632.0,30376.0,61.937854,24.576344,2021-07-17,3.0,0.0,8.142857,0.000000,"68,59",40.25
147,ADAMANTINA,3500105.0,35111.0,21747.0,7997.0,632.0,30376.0,61.937854,24.576344,2021-07-18,5.0,0.0,7.714286,0.000000,"65,58",45.00
